## Распознование текста из микрофона

In [24]:
!py3_env/bin/pip install SpeechRecognition
!py3_env/bin/pip install PyAudio

In [16]:
import speech_recognition as sr

In [52]:
recognizer = sr.Recognizer()

In [13]:
recognizer.listen??

In [53]:
with sr.Microphone() as micro:
    print(1)
    audio = recognizer.listen(micro)
    print(2)

1
2


In [54]:
with open('data/microphone-results.wav', "wb") as f:
    f.write(audio.get_wav_data())

In [56]:
recognizer.recognize_google(audio, language='en-EN')

'this is a sound detection chick 111 detection check'

## Распознование текста из аудиофайла

In [31]:
recognizer = sr.Recognizer()

In [58]:
with sr.AudioFile('data/microphone-results.wav') as source:
    audio1 = recognizer.record(source)

In [59]:
recognizer.recognize_google(audio1, language='en-EN')

'this is a sound detection chick 111 detection check'

## Text topic extraction

In [36]:
!py3_env/bin/pip install spacy
!py3_env/bin/pip install pymorphy2

In [37]:
import spacy

In [39]:
# spacy.load('ru')

In [60]:
from spacy.lang.en import English

In [61]:
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [63]:
text = recognizer.recognize_google(audio, language='en-EN')
tokenize(text)

['this', 'is', 'a', 'sound', 'detection', 'chick', '111', 'detection', 'check']

In [46]:
!py3_env/bin/pip install nltk

In [47]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/anton-
[nltk_data]     taleckij/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [48]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [78]:
for token in tokenize(text):
    print(get_lemma(token), get_lemma2(token))

this this
be is
a a
sound sound
detection detection
chick chick
111 111
detection detection
check check


In [67]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/anton-
[nltk_data]     taleckij/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [72]:
list(en_stop)[:5]

['herself', 'such', 'here', 'y', 'on']

In [90]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    return [get_lemma(token) for token in tokens 
            if len(token) > 4 and token not in en_stop]

In [91]:
text_data = [prepare_text_for_lda(text)]
text_data

[['sound', 'detection', 'chick', 'detection', 'check']]

In [82]:
!py3_env/bin/pip install gensim

In [86]:
from gensim import corpora

dictionary = corpora.Dictionary(text_data)
print(dictionary)
corpus = [dictionary.doc2bow(text) for text in text_data]
print(corpus)

Dictionary(4 unique tokens: ['check', 'chick', 'detection', 'sound'])
[[(0, 1), (1, 1), (2, 2), (3, 1)]]


In [89]:
dictionary.doc2bow?? # document to bag of words

In [88]:
import gensim

NUM_TOPICS = 2
NUM_WORDS = 3

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15)

topics = ldamodel.print_topics(num_words=NUM_WORDS)
for topic in topics:
    print(topic)

(0, '0.251*"detection" + 0.250*"check" + 0.250*"sound"')
(1, '0.359*"detection" + 0.214*"chick" + 0.214*"sound"')
